# Estimating Parameter Confidence Intervals With Bootstrapping
This notebook demonstrates the calculations required to do confidence interval constructions.
1. Construct a good model. This means checking that we get good $R^2$ values (or other model quality metrics) for each fold in a cross validation.
1. Compute residuals for the good model.
1. Construct a collection of parameter estimates. That is, for many repetitions
   1. Construct new observations (by using randomly selected residuals)
   1. Estimate parameter values
1. Compute the mean and standard deviation of the parameter estimates
1. Construct the confidence interval

In [1]:
%matplotlib inline
import tellurium as te
import numpy as np
import lmfit   # Fitting lib
import math
import random 
import matplotlib.pyplot as plt
import model_fitting as mf

In [2]:
# Model used in this example
model = """
     A -> B; k1*A
     B -> C; k2*B
      
     A = 5;
     B = 0;
     C = 0;
     k1 = 0.1
     k2 = 0.2
"""
parameters = mf.makeParameters(constants=['k1', 'k2'])

In [3]:
# Globals
num_points = 15
sim_time = 30
nose_std = 0.5

In [4]:
# Create synthetic observational data for this example. This is for demonstration purposes only.
# In practice, you will have observational data from experiments.
obs_data = mf.makeObservations(model=model, noise_std=nose_std, num_points=num_points, sim_time=sim_time)

## Step 1: Construct a good model.
In the following, we use the same model as the synthetic observations. Of course, in practice, you won't know the "true" model. You'll try many, and choose the best in terms of the quality metrics (e.g., $R^2$).

In [5]:
# Do the cross validation for this model. the crossValidate function returns two values: list of
# the parameters (for each fold) and RSQs for each fold.
list_parameters, rsqs = mf.crossValidate(obs_data, model=model, parameters=parameters, num_points=num_points, 
                                         sim_time=sim_time,
                                         num_folds=3)
rsqs

[0.937396238214605, 0.9747393871703416, 0.9385443584234233]

These are very good $R^2$ values. So, we accept the model.

Next, we need to estimate the parameter values to use in our model. To this end, we do a fit on the full set of data.

In [6]:
fitted_parameters = mf.fit(obs_data, model=model, parameters=parameters, num_points=num_points, sim_time=sim_time)
fitted_parameters

name,value,standard error,relative error,initial value,min,max,vary
k1,0.10605294,0.00548068,(5.17%),1,0.00000000,10.0000000,True
k2,0.17615774,0.01480438,(8.40%),1,0.00000000,10.0000000,True


# Step 2: Compute the Residuals

In [7]:
data = mf.runSimulation(model=model, parameters=fitted_parameters, num_points=num_points, sim_time=sim_time)
residuals = mf.calcSimulationResiduals(fitted_parameters, obs_data, num_points=num_points, sim_time=sim_time)
residuals

[ 0.85746807  0.          0.          0.42447258 -0.09008738  0.01017669
 -0.3473182  -0.35585509  0.0272486  -0.52173537  0.03422104  0.04078279
  0.27014028 -0.29779526  0.04786396  0.99524236  0.00203976  0.04888016
  0.09200861  0.76294564  0.04541438 -0.06911266  0.15713272  0.03919944
  0.0535522  -0.22274096  0.03170087 -0.28889839 -0.62775428  0.02399323
 -0.51523923  0.32630889  0.0167778  -0.4104997   0.19218073  0.01045098
  0.01620198  0.54235451  0.00518566 -0.26056776 -0.2375908   0.00100384
  0.2752238  -0.20179099 -0.0021656 ]

In [9]:
# The standard deviation of the residuals should be approximately the same as the standard deviation
# of the random noise we injected in the construction of the observations.
np.std(residuals)

0.3284225196989527

## Step 3: Construct New Observations